In [48]:
import numpy as np
from pathlib import Path
import pandas as pd
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator


datapath = Path.cwd().parent / "data" / "eval" / "1-eval_cls_perturb.csv"
data = pd.read_csv(datapath)

# drop unnecessary cols
data = data.drop(columns=['img_id', 'label'])

# drop multiple rows where perturb=False but epsilon values are different
filtered_data = data[data['perturb'] == False]
result = filtered_data.groupby('epsilon').first().reset_index()

# drop all perturb=False rows in original and replace with new rows
len_before = len(data)
data = data[data['perturb'] == True]
data = pd.concat([data, result], ignore_index=True)
len_after = len(data)
print(f"removed {len_before - len_after} rows with perturb=False and different epsilon values")
# display(HTML(data.groupby(["mask", "opacity", "density", "perturb", "epsilon"]).mean().reset_index().to_html()))


# add util cols
data['adv_acc'] = (data['advx_acc1'] + data['advx_acc5']) / 2
data['quality'] = (data['cosine_sim'] + data['psnr'] / 100 + data['ssim']) / 3

plt.figure(figsize=(7, 25))

markers = {'square': 's', 'diamond': 'D'}
colors = {64: 'lightblue', 128: 'skyblue', 192: 'deepskyblue', 255: 'blue'}

for mask in markers:
    for opacity in colors:
        for perturb in [True, False]:
            mask_opacity_perturb_data = data[(data['mask'] == mask) & 
                                             (data['opacity'] == opacity) & 
                                             (data['perturb'] == perturb)]
            
            for _, row in mask_opacity_perturb_data.iterrows():
                plt.scatter(row['adv_acc'], row['quality'], 
                            marker=markers[mask], 
                            c=colors[opacity], 
                            s=row['density']*10,
                            alpha=0.7 if perturb else 0.3,
                            edgecolors='black' if perturb else 'gray')
                
                # add epsilon value as text next to each point
                plt.annotate(f"epsilon: {row['epsilon']}", 
                             (row['adv_acc'], row['quality']), 
                             xytext=(20, 3), 
                             textcoords='offset points', 
                             fontsize=8)

plt.xlabel('Adversarial Accuracy (lower is better)')
plt.ylabel('Quality Metric (higher is better)')
plt.title('Trade-off between Adversarial Accuracy and Quality')

legend_elements = [
    plt.Line2D([0], [0], marker='s', color='w', label='Square', markerfacecolor='gray', markersize=10),
    plt.Line2D([0], [0], marker='D', color='w', label='Diamond', markerfacecolor='gray', markersize=10),
    plt.Line2D([0], [0], marker='o', color='lightblue', label='Opacity 64', markersize=10),
    plt.Line2D([0], [0], marker='o', color='skyblue', label='Opacity 128', markersize=10),
    plt.Line2D([0], [0], marker='o', color='deepskyblue', label='Opacity 192', markersize=10),
    plt.Line2D([0], [0], marker='o', color='blue', label='Opacity 255', markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', markeredgecolor='black', label='Perturb True', markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', markeredgecolor='gray', label='Perturb False', markersize=10, alpha=0.3),
    plt.Line2D([0], [0], marker='o', color='w', label='Density', markerfacecolor='gray', markersize=5),
    plt.Line2D([0], [0], marker='o', color='w', label='', markerfacecolor='gray', markersize=10)
]

plt.legend(handles=legend_elements, title='Legend', loc='center left', bbox_to_anchor=(1, 0.5))

plt.tight_layout()
plt.show()

# -----

top_low_adv = data.nsmallest(10, 'adv_acc').nlargest(10, 'quality')
top_low_adv = top_low_adv.reset_index(drop=True)
top_low_adv.index += 1

print("Top 10 results (lowest adversarial accuracy and highest quality):")
print(top_low_adv[['mask', 'opacity', 'density', 'perturb', 'epsilon', 'adv_acc', 'quality']])

removed 714 rows with perturb=False and different epsilon values
